In [1]:
# https://towardsdatascience.com/text-summarization-using-deep-neural-networks-e7ee7521d804/  
https://saadsohail5104.medium.com/understanding-padding-in-nlp-types-and-when-to-use-them-bacae6cae401
https://github.com/rohithreddy024/Text-Summarizer-Pytorch/blob/master/data_util/data.py
https://www.ijcaonline.org/archives/volume185/number15/patil-2023-ijca-922837.pdf
https://www.kaggle.com/code/getanmolgupta01/text-summarization-lstm-encoder-decoder
https://github.com/rohithreddy024/Text-Summarizer-Pytorch/blob/master/model.py
https://arxiv.org/abs/1508.04025
https://www.freecodecamp.org/news/what-is-rouge-and-how-it-works-for-evaluation-of-summaries-e059fb8ac840/#:~:text=ROUGE%20stands%20for%20Recall%2DOriented,as%20well%20as%20machine%20translations.&text=If%20we%20consider%20just%20the,and%20reference%20summary%20is%206.
https://www.kaggle.com/code/quadeer15sh/introduction-to-language-modelling-using-rnns
https://www.kaggle.com/code/divyanshvishwkarma/seq2seq-model-with-attention-teacher-forcing-tf # nguồn data

SyntaxError: invalid decimal literal (2005343661.py, line 4)

In [ ]:
class SimpleEncoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=HIDDEN_DIM):
        super().__init__()
        self.embedding = embedding_layer  # from_pretrained
        self.lstm = nn.LSTM(
            input_size=self.embedding.embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True,
            bidirectional=False  # Unidirectional
        )
    
    def forward(self, x, seq_lens):
        x = self.embedding(x)
    
        packed = pack_padded_sequence(input=x,lengths=seq_lens.cpu(),batch_first=True, enforce_sorted=False  
        )
        
        packed_output, (hidden, cell) = self.lstm(packed)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        
        return output, (hidden, cell)  # hidden shape: [1, batch_size, hidden_dim]
    
class SimpleAttention(nn.Module):
    def __init__(self, hidden_dim=HIDDEN_DIM):
        super().__init__()
        self.energy = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),  # Nhận input là [encoder_output + decoder_hidden]
            nn.Tanh(),                      
            nn.Linear(hidden_dim, 1, bias=False)    #  Trả về 1 chiều attention scores
        )
    
    def forward(self, decoder_hidden, encoder_outputs, mask=None):
        #########  decoder_hidden = decoder_hidden.unsqueeze(1).expand_as(encoder_outputs)  # [batch_size, seq_len, hidden_dim]
        decoder_hidden = decoder_hidden.unsqueeze(1)  # [batch_size, 1, hidden_dim]
        decoder_hidden = decoder_hidden.repeat(1, encoder_outputs.size(1), 1)

        combined = torch.cat([encoder_outputs, decoder_hidden], dim=2)  # [batch_size, seq_len, hidden_dim * 2]
        scores = self.energy(combined).squeeze(2)  # [batch_size, seq_len]
        
        # 3: Áp dụng mask và softmax ( is it necessary)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e10)
        attn_weights = F.softmax(scores, dim=1)    # [batch_size, seq_len]
        
        #  context = [batch_size, 1, hidden_dim]squeeze(1) loại chiều 1 -> [batch_size, hidden_dim]
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1) 
    
        return context, attn_weights
class SimpleDecoder(nn.Module):
    def __init__(self, embedding_layer, hidden_dim=HIDDEN_DIM, vocab_size=vocab_size):
        super().__init__()
        self.embedding = embedding_layer
        self.lstm = nn.LSTMCell(
            input_size=self.embedding.embedding_dim + hidden_dim,  # Add context_dim
            hidden_size=hidden_dim
        )
        self.attention = SimpleAttention(hidden_dim)
        self.fc_out = nn.Linear(hidden_dim * 2, vocab_size)  # Combine hidden + context

    def forward(self, x, prev_hidden, prev_cell, encoder_outputs, mask=None):
        x = self.embedding(x)  # [batch_size] -> [batch_size, emb_dim]
        
        if prev_hidden.dim() == 3:  # If it's [num_layers, batch_size, hidden_dim]
            prev_hidden = prev_hidden[-1]  # Take last layer's hidden state
        
        context, attn_weights = self.attention(prev_hidden, encoder_outputs, mask)
        
        # Combine embedding and context to get input for LSTM
        lstm_input = torch.cat([x, context], dim=1)  # [batch_size, emb_dim + hidden_dim]
        
        hidden, cell = self.lstm(lstm_input, (prev_hidden, prev_cell))
        
        # Kết hợp hidden và context để dự đoán từ
        output_input = torch.cat([hidden, context], dim=1)
        output = self.fc_out(output_input)
        
        return output, hidden, cell, attn_weights
    
class Seq2SeqModel(nn.Module):
    def __init__(self, embedding_layer, hidden_dim, vocab_size):
        super().__init__()
        self.encoder = SimpleEncoder(embedding_layer, hidden_dim)
        self.decoder = SimpleDecoder(embedding_layer, hidden_dim, vocab_size)
        self.vocab_size = vocab_size
        self.start_id = stoi("<SOS>")  # Thêm start token ID

    def forward(self, src, src_lens, trg=None, max_len=MAX_LENGTH_SUMMARY, teacher_forcing_ratio=0.5):
        enc_outputs, (hidden, cell) = self.encoder(src, src_lens)
        
        batch_size = src.size(0)
        if trg is None:
            max_len = max_len
            trg = torch.full((batch_size,), self.start_id, dtype=torch.long, device=src.device)
        else:  # Training mode
            max_len = trg.size(1)

        outputs = torch.zeros(batch_size, max_len, self.vocab_size).to(src.device)

        x = torch.full((batch_size,), self.start_id, dtype=torch.long, device=src.device)  # Đổi tên x_t -> x
        
        # Squeeze the encoder hidden states for the decoder
        hidden = hidden.squeeze(0)  # [1, batch_size, hidden_dim] -> [batch_size, hidden_dim]
        cell = cell.squeeze(0)
        
        for t in range(max_len):
            output, hidden, cell, _ = self.decoder(
                x=x,
                prev_hidden=hidden,
                prev_cell=cell,
                encoder_outputs=enc_outputs
            )
            outputs[:, t] = output
            
            if trg is not None and random.random() < teacher_forcing_ratio:
                x = trg[:, t]
            else:
                x = output.argmax(1)
        
        return outputs